# Sora2 Watermark Remover - Simple Version

シンプルな6ステップでSora動画からウォーターマークを除去します。

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. 依存関係のインストール

In [ ]:
!pip install -q iopaint transformers

## 2. LaMaモデルのダウンロード

In [ ]:
!iopaint download --model lama

## 3. 最新コードの取得

In [ ]:
!curl -sL https://github.com/fulfulggg/Sora2WatermarkRemover/archive/refs/heads/main.tar.gz | tar xz --strip-components=1
print("\n✅ リポジトリのダウンロードが完了しました。")

## 4. 動画のアップロード

In [ ]:
from google.colab import files

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

input_file = list(uploaded.keys())[0]
print(f"\n✅ アップロード完了: {input_file}")

## 5. ウォーターマークの除去（AI検出）

Florence-2 AIがウォーターマークを自動検出して除去します。

**パラメータ調整（必要に応じて）:**
- `--max-bbox-percent`: 検出サイズの上限（デフォルト: 15%）
- `--frame-step`: 処理間隔（1=全フレーム、2=隔フレーム）
- `--target-fps`: 出力fps（0=入力と同じ）

In [ ]:
!python remwm.py "{input_file}" "/content/out.mp4" \
  --max-bbox-percent=15 \
  --frame-step=1 \
  --target-fps=0 \
  --force-format=MP4 \
  --overwrite

## 6. 結果のダウンロード

In [ ]:
print("結果をダウンロード中...")
files.download('/content/out.mp4')
print("\n✅ 完了！")

---

## トラブルシューティング: ウォーターマークが残っている場合

**⚠️ 通常は上記のステップ1-6だけでOKです。**

一部フレームにウォーターマークが残っている場合のみ、以下を使用してください。

### 方法1: 固定位置の場合

ウォーターマークが常に同じ位置にある場合

In [ ]:
# ステップ1: プレビュー生成（枠表示で位置確認）
# x,y,w,h を調整してウォーターマークを完全に覆うようにする
!ffmpeg -y -i "{input_file}" -vf "delogo=x=80:y=80:w=320:h=120:show=1" -t 3 /content/preview.mp4

print("\n✅ プレビュー生成完了")
print("次のセルでダウンロードして、枠の位置を確認してください")

In [ ]:
# プレビューをダウンロード
files.download('/content/preview.mp4')
print("\n📥 ダウンロード完了！枠がウォーターマークを覆っているか確認してください")
print("覆っていない場合は、上のセルのx,y,w,hを調整して再実行してください")

In [ ]:
# ステップ2: 本番処理（上で確認した座標を使用）
!ffmpeg -y -i "{input_file}" -vf "delogo=x=80:y=80:w=320:h=120" -c:a copy /content/out_fix.mp4
print("\n✅ 処理完了！")

In [ ]:
# 結果をダウンロード
files.download('/content/out_fix.mp4')
print("\n✅ 完了！")

### 方法2: 位置が時間で変わる場合

ウォーターマークの位置が時間と共に変わる場合、時間区間ごとに座標を指定します。

**例:** 0-2.2秒は左上、2.2-5秒は右下

In [ ]:
# 時間区間を指定して処理
# enable='between(t,開始秒,終了秒)' で時間を指定
!ffmpeg -y -i "{input_file}" -vf "delogo=x=80:y=80:w=320:h=120:enable='between(t,0,2.2)',delogo=x=520:y=840:w=360:h=140:enable='between(t,2.2,5.0)'" -c:a copy /content/out_fix.mp4
print("\n✅ 処理完了！")

In [ ]:
# 結果をダウンロード
files.download('/content/out_fix.mp4')
print("\n✅ 完了！")